In [ ]:
#  Code ref: https://ritscm.regeneron.com/projects/DSEAIM/repos/secureaccess/browse/securekeyapp.py

In [ ]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org --upgrade pip

In [ ]:
!pip install  --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org \
msal \
retry \
simplejson \
openai \
grpcio==1.58.0 \
pymilvus==2.3.5 \
protobuf \
grpcio-tools==1.58.0

In [ ]:
import os
import openai
import requests
import simplejson as json
from retry import retry
from msal import PublicClientApplication, ConfidentialClientApplication, ClientApplication
import time

In [ ]:
@retry (tries=3, delay=2)
def getapikey():
    # These are the AZURE parameters needed by the client application
    # In this scheme, the user does not have to worry about the api key, all of that is handled
    # at the AZURE Back End.   The API key is rotated hourly.
    # We can handle these as Kuberbetes secrets, or as environment variables.
    client_id = os.getenv("AZURE_CLIENT_ID")
    tenant_id = os.getenv("AZURE_TENANT_ID")
    endpoint = os.getenv("AZURE_ENDPOINT")

    scopes = [os.getenv("AZURE_APPLICATION_SCOPE")]

    app = ClientApplication(
        client_id=client_id,
        authority="https://login.microsoftonline.com/" + tenant_id
    )


    acquire_tokens_result = app.acquire_token_by_username_password(username=os.getenv("SVC_ACCOUNT"),
                                                                   password=os.getenv("SVC_PASSWORD"),
                                                                   scopes=scopes)
    if 'error' in acquire_tokens_result:
        print("Error: " + acquire_tokens_result['error'])
        print("Description: " + acquire_tokens_result['error_description'])
        return 2
    else:
        header_token = {"Authorization": "Bearer {}".format(acquire_tokens_result['access_token'])}
        rt = requests.post(url=endpoint, headers=header_token, data=b'{"key":"openaikey2"}')
        return rt.json()

In [ ]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
# We are dynamically getting the key from AZURE>   Access is based on the service account/ad group combination
openai.api_key = getapikey()
print(openai.api_key)

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = getapikey(),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

protein = "{\"_id\":\"582b467fbd364f2b2a00009c\",\"_type\":\"Protein\",\"associated_plasmid_regn_names\":[],\"created_at\":\"2016-11-15T17:31:43.154Z\",\"created_by\":\"kristin.hudson\",\"eln_ref\":\"https://researcheln.regeneron.regn.com:8443/EWorkbookWebApp/#entity/displayEntity?entityId=e4cd7280ab5811e6a49840a8f01e721c\\u0026r=y\",\"linked_pes_prgns\":null,\"pes_created_at\":null,\"pes_created_by\":null,\"project_numbers\":[],\"protein_chain_ids\":[\"582b45c777ab416e41000031\",\"582b462ebd364f2b2a00009b\"],\"protein_sources\":[{\"_id\":\"5f7e341e7baad70118845ce8\",\"created_at\":\"2020-10-07T21:33:18.831Z\",\"regn_name\":\"PN22022\",\"updated_at\":\"2020-10-07T21:33:18.831Z\"}],\"protein_type\":{\"_id\":\"5d07a07a700f3e489862adc1\",\"chain_structure\":[2,2],\"chains\":4,\"created_at\":\"2016-11-09T19:27:50.386Z\",\"name\":\"Standard mAb\",\"protein_category\":\"Common Antibody \\u0026 Antibody-like Formats\",\"protein_id\":null,\"svg\":\"data:image/svg+xml;base64, IDxzdmcgdmVyc2lvbj0iMS4xIiBpZD0iTGF5ZXJfMSIgeG1sbnM9Imh0dHA6\\nLy93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3\\ndy53My5vcmcvMTk5OS94bGluayIgeD0iMHB4IiB5PSIwcHgiIHdpZHRoPSIz\\nODhweCIgaGVpZ2h0PSIzNTkuMjI3cHgiIHZpZXdCb3g9IjAgMCAzODggMzU5\\nLjIyNyIgZW5hYmxlLWJhY2tncm91bmQ9Im5ldyAwIDAgMzg4IDM1OS4yMjci\\nIHhtbDpzcGFjZT0icHJlc2VydmUiPiA8Zz4gPGc+IDxyZWN0IHg9IjE5Ni41\\nIiB5PSIxNTYuOTc3IiBmaWxsPSIjQ0MwMDAwIiB3aWR0aD0iMjYuMTU3IiBo\\nZWlnaHQ9IjE4NC41Ii8+IDwvZz4gPC9nPiA8Zz4gPGc+IDxyZWN0IHg9IjE2\\nNy41IiB5PSIxNTYuOTc3IiBmaWxsPSIjQ0MwMDAwIiB3aWR0aD0iMjYuMTU3\\nIiBoZWlnaHQ9IjE4NC41Ii8+IDwvZz4gPC9nPiA8cmVjdCB4PSIzNTEuNSIg\\neT0iMzI3LjcyNyIgZmlsbD0iI0NDMDAwMSIgd2lkdGg9IjEiIGhlaWdodD0i\\nMCIvPiA8Zz4gPHJlY3QgeD0iMjY1LjU2MSIgeT0iLTkuNzUyIiB0cmFuc2Zv\\ncm09Im1hdHJpeCgwLjcwNzEgMC43MDcxIC0wLjcwNzEgMC43MDcxIDEzOS45\\nNDY3IC0xNzIuODY1MikiIGZpbGw9IiNDQzAwMDAiIHdpZHRoPSIyNi4xNTci\\nIGhlaWdodD0iMTg0LjUiLz4gPC9nPiA8Zz4gPHJlY3QgeD0iOTguNDM5IiB5\\nPSItOS43NTIiIHRyYW5zZm9ybT0ibWF0cml4KC0wLjcwNzEgMC43MDcxIC0w\\nLjcwNzEgLTAuNzA3MSAyNDguNzA3NiA2MS45ODA5KSIgZmlsbD0iI0NDMDAw\\nMCIgd2lkdGg9IjI2LjE1NyIgaGVpZ2h0PSIxODQuNSIvPiA8L2c+IDxnPiA8\\ncmVjdCB4PSI3Ni43OTMiIHk9IjExLjc4NCIgdHJhbnNmb3JtPSJtYXRyaXgo\\nLTAuNzA3MSAwLjcwNzEgLTAuNzA3MSAtMC43MDcxIDIyNi43OTU2IDExMy41\\nOTgpIiBmaWxsPSIjMDAwMEZGIiB3aWR0aD0iMjYuMTU3IiBoZWlnaHQ9IjE4\\nMy45NyIvPiA8L2c+IDxnPiA8cmVjdCB4PSIyODguMjQ1IiB5PSIxMS43ODUi\\nIHRyYW5zZm9ybT0ibWF0cml4KC0wLjcwNzEgLTAuNzA3MSAwLjcwNzEgLTAu\\nNzA3MSA0NDEuMDE2MSAzOTAuMjEzOSkiIGZpbGw9IiMwMDAwRkYiIHdpZHRo\\nPSIyNi4xNTciIGhlaWdodD0iMTgzLjk3Ii8+IDwvZz4gPC9zdmc+IA==\\n\",\"unique_chains\":2,\"updated_at\":\"2016-11-09T19:27:50.386Z\"},\"purposes\":[\"Isotype Control\"],\"pushed_to_lims\":true,\"regn_name\":\"REGN5012\",\"regn_number\":5012,\"resolved_at\":\"2016-11-15T17:31:43.154Z\",\"resolved_by\":\"kristin.hudson\",\"synonyms\":[{\"_id\":\"5b6020e3700f3e449e654a63\",\"type\":\"abpid\",\"value\":\"H4H22022N\"}],\"updated_at\":\"2023-04-21T09:45:56.211Z\"}"

response = client.embeddings.create(
    input = protein,
    model= "RegnADA002"
)

In [ ]:
print(f"JSON string length: {len(protein)}")
len(response.data[0].embedding)

In [ ]:
from pymilvus import connections, utility, db
conn = connections.connect(host="standalone", port=19530, db_name="default")
db.list_database()

In [ ]:
found = False
for database_name in db.list_database():
    if database_name == "bioregistry":
        found = True
if not found:
    db.create_database("bioregistry")

In [ ]:
db.list_database()

In [ ]:
db.using_database("bioregistry")

In [ ]:
utility.list_collections()

In [ ]:
new_collection_name = "proteins"

In [ ]:
for collection_name in utility.list_collections():
    if collection_name == new_collection_name:
        utility.drop_collection(new_collection_name)

In [ ]:
from pymilvus import CollectionSchema, FieldSchema, DataType

protein_id = FieldSchema(
  name="protein_id",
  dtype=DataType.INT64,
  is_primary=True,
)

name = FieldSchema(
  name="name",
  dtype=DataType.VARCHAR,
  max_length=32,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)

char_count = FieldSchema(
  name="char_count",
  dtype=DataType.INT64,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value=9999
)

protein_json = FieldSchema(
  name="protein_json",
  dtype=DataType.VARCHAR,
  max_length=4000,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)

protein_vector = FieldSchema(
  name="protein_vector",
  dtype=DataType.FLOAT_VECTOR,
  dim=1536
)

schema = CollectionSchema(
  fields=[protein_id, name, char_count, protein_json, protein_vector],
  description="Protein search",
  enable_dynamic_field=True
)

In [ ]:
from pymilvus import Collection
collection = Collection(
    name=new_collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )

In [ ]:
collection.create_partition("Disc1")
collection.has_partition("Disc1")

In [ ]:
collection.create_partition("Disc2")
collection.has_partition("Disc2")

In [ ]:
from pymilvus import Partition
disc2 = Partition(collection, name="Disc2")
disc2.release()
collection.drop_partition("Disc2")
collection.has_partition("Disc2")

In [ ]:
import json
protein_obj = json.loads(protein)
protein_obj['regn_name']

In [ ]:
utility.list_collections()

In [ ]:
# Insert one vetcor
# { protein_id, name, char_count, protein_json, protein_vector }
protein_id_column = [1]
name_column = [protein_obj['regn_name']]
char_count_column = [len(protein)]
protein_json_column = [protein]
protein_vector_column = [response.data[0].embedding]

In [ ]:
data = [protein_id_column, name_column, char_count_column, protein_json_column, protein_vector_column]

In [ ]:
mr = collection.upsert(data)

In [ ]:
mr

In [ ]:

collection.compact()

In [ ]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_SQ8",
  "params":{"nlist":1024},
  
}
collection.create_index(
  field_name="protein_vector", 
  index_name="protein_vector_idx",
  index_params=index_params
)

In [ ]:
# index on a scalar field
collection.create_index(
  field_name="name", 
  index_name="protein_name_idx"
)

In [ ]:
# Search , Query and Hybrid

In [ ]:
search_string = "kristin.hudson" 
response = client.embeddings.create(
    input = search_string,
    model= "RegnADA002"
)
query_embedding = response.data[0].embedding
query_embedding

In [ ]:
search_params = {
    "metric_type": "L2", 
    "params": {"search_k": 64}
}

In [ ]:
# Search
collection = Collection(new_collection_name)
utility.list_collections()
collection.load()
results =  collection.search(
    data=[query_embedding],
    anns_field="protein_vector",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['protein_json']
)
for result in results[0]:
    print(result)

In [ ]:
query_expr = f"name == '{protein_obj['regn_name']}'"

In [ ]:
# Query
collection = Collection(new_collection_name)
utility.list_collections()
collection.load()
results = collection.query(
    expr = query_expr,
    offset = 0,
    limit = 10, 
    output_fields = ["name", "protein_json"]
)
results[0]

In [ ]:
# Hybrid : sama as search with expr provided
results =  collection.search(
    data=[query_embedding],
    anns_field="protein_vector",
    param=search_params,
    limit=5,
    expr=query_expr,
    output_fields=['protein_json']
)
for result in results[0]:
    print(result)